# Import necessary packages

In [ ]:
import numpy as np
import sep

In [ ]:
##additional setup for reading the test image and displaying plots
import astropy.io.fits as fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

rcParams['figure.figsize'] = [10.,8.]

### Read image into standard 2-d numpy array

In [ ]:
data = fits.getdata("image.fits")

### Show the image

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data,interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('astro_image_1.png', bbox_inches = 'tight', dpi=300)

# Background Subtraction

Most optical/IR data must be background subtracted before sources can be deteced. In SEP, background esetimation and source decetion are two seperate steps.

In [ ]:
# measure a spatially varying background on the image

bkg = sep.Background(data)


Control box size used in estimating background. It is also possible to mask pixels

We now have an object (Background) that holds ingo on the spatially varying background and spatially varying background noise level. We can now do various things with this (Background) object

In [ ]:
#get a 'global' mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
### These numbers are different than the tutorials because I'm
### Using the Hubble image provided my prof instead of the tutorial image

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
#bkg_image = np.array(bkg) # equivalent to above

In [ ]:
# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray',origin='lower')
plt.colorbar()
plt.savefig('astro_image_2.png', bbox_inches = 'tight', dpi=300)

In [ ]:
# evaluate the background noise as a 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar()
plt.savefig('astro_image_3.png', bbox_inches = 'tight', dpi=300)

In [ ]:
# subtract the background
data_sub = data - bkg

# Object Detection

Background noise level is too flat, we need to set the detection threshold to be a constant value of sigma, where sigma is background RMS



In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
# how many objexts were detected
len(objects)

Objects['x'] and objects['y'] give the centroid coordinates of the objects. Now we want to check where the objects are located while putting some basic parameters on the image

In [ ]:
from matplotlib.patches import Ellipse

#plot background subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s,
                vmax=m+s, origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i],objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] *180./np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('astro_image_4.png', bbox_inches = 'tight', dpi=300)

In [ ]:
# see available fields of objects
objects.dtype.names

# Aperture Photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub,objects['x'], objects['y'],
                                    3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 19 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i,flux[i], fluxerr[i]))